In [ ]:
import json 
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

with open("C:/Users/david/Desktop/Ongoing Projects/uw-upnext/public/events.json") as f:
    events = json.load(f)

In [ ]:
# Load into DataFrame
df = pd.DataFrame(events)
print(df.columns)
# Filter for is_event == True

df = df[df["event_details"].apply(lambda x: x["is_event"] == True)]


In [ ]:
df["event_name"] = df["event_details"].apply(lambda x: x["event_name"])
df = df.drop_duplicates(subset=["event_name"])
df.drop(columns=["event_name"])

,_id,url,likes,display_photo,account,date,caption,accessibility_caption,embedded,event_details
0,6727e7fb7d33b8a433a8fe72,DBuqRziA-Zc,268.0,https://instagram.fyzd1-3.fna.fbcdn.net/v/t51....,uwcsa,2024-10-30 01:00:35,🐺🩸BLOOD SWEAT & FEARS 2.0🩸🐺 ‼️TICKETS ARE SOLD...,None,,"{'is_event': True, 'event_name': 'BLOOD SWEAT ..."
2,6727e7fe7d33b8a433a8fe74,DAM_kHWA1DG,400.0,https://instagram.fyzd1-2.fna.fbcdn.net/v/t51....,uwcsa,2024-09-22 02:41:01,DONT FORGET MON. SEPT 23rd AT COLUMBIA LAKE FI...,None,,"{'is_event': True, 'event_name': 'UWC 50th Ann..."
3,6727e7ff7d33b8a433a8fe75,C_9arvnA3k0,422.0,https://instagram.fyzd1-2.fna.fbcdn.net/v/t51....,uwcsa,2024-09-16 01:30:08,✨🌕UWCSA’s Mid-Autumn Bonfire Festival is HERE!...,None,,"{'is_event': True, 'event_name': 'UWCSA's Mid-..."
4,6727e8007d33b8a433a8fe76,C92jhFzA5hh,204.0,https://instagram.fyzd1-2.fna.fbcdn.net/v/t51....,uwcsa,2024-07-25 16:29:51,The Night Market is happening tonight at 6:30p...,None,,"{'is_event': True, 'event_name': 'The Night Ma..."
5,6727e8027d33b8a433a8fe77,C9yGoxXyj3i,315.0,https://instagram.fyzd1-2.fna.fbcdn.net/v/t51....,uwcsa,2024-07-23 23:00:00,What’s that I smell in the air 😋? CSA is colla...,None,,"{'is_event': True, 'event_name': 'CSA Night Ma..."
...,...,...,...,...,...,...,...,...,...,...
1077,6729a16f38120b04865494eb,C9odA_SNnym,31.0,https://instagram.fyzd1-3.fna.fbcdn.net/v/t39....,uw_bmsa,2024-07-20 05:03:40,Exams season is coming up… so come join us at ...,None,,"{'is_event': True, 'event_name': 'UW BMSA Stud..."
1079,6729a17238120b04865494ed,C8DHdjby0f8,3.0,https://instagram.fyzd1-3.fna.fbcdn.net/v/t39....,uw_bmsa,2024-06-10 20:32:02,💫Join us for an afternoon of creativity and ex...,None,,"{'is_event': True, 'event_name': 'BMSA Paint S..."
1080,6729a17438120b04865494ee,C5NPMgQAWbd,47.0,https://instagram.fyzd1-3.fna.fbcdn.net/v/t51....,uw_bmsa,2024-04-01 05:17:50,Student-Prof night is fast approaching! If you...,None,,"{'is_event': True, 'event_name': 'Student-Prof..."
1081,6729a17538120b04865494ef,C4QbNvrro5a,35.0,https://instagram.fyzd1-3.fna.fbcdn.net/v/t39....,uw_bmsa,2024-03-08 14:29:12,Join us on March 13 (5:30-6:30PM) in STC 0020 ...,None,,"{'is_event': True, 'event_name': 'Genevieve Me..."


In [16]:
from typing import List
from together import Together
import time

together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'BAAI/bge-large-en-v1.5' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

for index, row in df.iterrows():
    row_dict = {}
    for column in df.columns.tolist():
        if column in ['account','date','caption','accessibility_caption','url','likes','display_photo']:
            row_dict[column] = row[column]
    row_dict["embedded"] = generate_embedding([f"Account: {str(row["account"])}, Date: {str(row['date'])}, Caption: {str(row["caption"])[:1000] + str(row["accessibility_caption"])}"], embedding_model_string)  
    row_dict["event_details"] = row["event_details"]
    result = collection.insert_one(row_dict)
    print(f"Inserted document ID: {result.inserted_id}")

        

Inserted document ID: 68d196a7b86242af9d164734
Inserted document ID: 68d196a7b86242af9d164735
Inserted document ID: 68d196a8b86242af9d164736
Inserted document ID: 68d196a8b86242af9d164737
Inserted document ID: 68d196a9b86242af9d164738
Inserted document ID: 68d196a9b86242af9d164739
Inserted document ID: 68d196a9b86242af9d16473a
Inserted document ID: 68d196a9b86242af9d16473b
Inserted document ID: 68d196aab86242af9d16473c
Inserted document ID: 68d196aab86242af9d16473d
Inserted document ID: 68d196aab86242af9d16473e
Inserted document ID: 68d196aab86242af9d16473f
Inserted document ID: 68d196abb86242af9d164740
Inserted document ID: 68d196abb86242af9d164741
Inserted document ID: 68d196abb86242af9d164742
Inserted document ID: 68d196abb86242af9d164743
Inserted document ID: 68d196acb86242af9d164744
Inserted document ID: 68d196acb86242af9d164745
Inserted document ID: 68d196adb86242af9d164746
Inserted document ID: 68d196adb86242af9d164747
Inserted document ID: 68d196adb86242af9d164748
Inserted docu